In [0]:
import pandas as pd
from simpletransformers.classification import ClassificationModel
import pandas as pd

In [0]:
import torch
torch.cuda.is_available()

True

## Install dependencies

In [0]:
# pip install torch==1.2.0 torchvision==0.4.0 -f https://download.pytorch.org/whl/torch_stable.html
# !pip install simpletransformers
# !pip3 install torch torchvision

## Install Apex (required for Hugging Face library)

In [0]:
!git clone https://github.com/NVIDIA/apex

Cloning into 'apex'...
remote: Enumerating objects: 45, done.
remote: Counting objects: 100% (45/45), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 5576 (delta 16), reused 13 (delta 7), pack-reused 5531
Receiving objects: 100% (5576/5576), 13.37 MiB | 6.51 MiB/s, done.
Resolving deltas: 100% (3596/3596), done.


In [0]:
cd apex

/content/apex


In [0]:
!python setup.py install

## Load Dataset

In [0]:
train = pd.read_csv('/cleaned_tweets.csv', index_col=0)

In [0]:
sources_not_to_keep = ['dataset2_abhiraj', 'dataset3_abhiraj', 'dataset1_abhiraj']

print(train.shape)
train = train[train.Cleaned != 'None']
train = train[~train.Source.isin(sources_not_to_keep)]
train.dropna(axis = 0, inplace = True)
print(train.shape)

train= train.sample(frac=1).reset_index(drop=True)

train['Target'] = train['Target'].astype(int)
train_short = train[['Cleaned_stop_words', 'Target']]

train_split = train_short[0 : int(len(train_short)*0.8)]
val_split = train_short[int(len(train_short)*0.8):]

(28992, 5)
(16441, 5)


In [0]:
# Create a ClassificationModel
model = ClassificationModel('roberta', 'roberta-base', args={'overwrite_output_dir': True, 'num_train_epochs': 5}) # You can set class weights by using the optional weight argument

model.train_model(train_split, eval_df=val_split)

# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(val_split)

100%|██████████| 501200538/501200538 [00:42<00:00, 11878026.98B/s]


Converting to features started.



Selected optimization level O1:  Insert automatic casts around Pytorch functions and Tensor methods.

Defaults for this optimization level are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic


Running loss: 0.680135

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:91: UserWarning: Seems like `optimizer.step()` has been overridden after learning rate scheduler initialization. Please, make sure to call `optimizer.step()` before `lr_scheduler.step()`. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


Running loss: 0.663903Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 32768.0
Running loss: 1.094736Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 16384.0
Running loss: 0.231873Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 8192.0
Running loss: 0.890964Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 4096.0
Running loss: 0.247539

Running loss: 0.471553Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 2048.0
Running loss: 0.058491

Running loss: 0.143755Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 1024.0
Running loss: 0.006843

Running loss: 0.908627Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 1024.0
Running loss: 0.200110Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 512.0
Running loss: 0.017898

Running loss: 0.015846
Training of roberta model complete. Saved to outputs/.
Converting to features started.


In [0]:
result

{'eval_loss': 0.4224352008364733,
 'fn': 177,
 'fp': 166,
 'mcc': 0.7566327368047308,
 'tn': 2095,
 'tp': 851}